In [ ]:
import numpy as np
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/cat-in-the-dat-ii/train.csv')
test = pd.read_csv('../input/cat-in-the-dat-ii/test.csv')

In [ ]:
train.sort_index(inplace=True)
y_train = train['target']
x_train = train.drop(['target', 'id'], axis=1)

In [ ]:
test_id = test['id']
test.drop('id', axis=1, inplace=True)

In [ ]:
for col in x_train.columns:
    if x_train[col].isna().sum()>0:
        x_train.loc[x_train[col].isna(),col] = x_train[col][-x_train[col].isna()].sample(n= x_train[col].isna().sum()).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3)

In [ ]:
X_train.isna().sum()

In [ ]:
high_dim_cols=[]
low_dim_cols=[]

for col in X_train.columns:
    if X_train[col].nunique()<20:
        low_dim_cols.append(col)

for col in X_train.columns:
    if X_train[col].nunique()>20:
        high_dim_cols.append(col)

In [ ]:
print(low_dim_cols)
print(high_dim_cols)

In [ ]:
ohEncoder = ce.OneHotEncoder(cols=low_dim_cols)
X_train = ohEncoder.fit_transform(X_train)
X_test = ohEncoder.transform(X_test)

In [ ]:
hashingEncoder = ce.HashingEncoder(cols=high_dim_cols)
X_train = hashingEncoder.fit_transform(X_train)
X_test = hashingEncoder.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=2020, fit_intercept=True, penalty='none', verbose=0).fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

In [ ]:
test = ohEncoder.transform(test)
test = hashingEncoder.transform(test)

In [ ]:
pd.DataFrame({'id': test_id, 'target': clf.predict_proba(test)[:,1]}).to_csv('submission.csv', index=False)